In [2]:
pip install pandas nltk

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

df1 = pd.read_csv('Datathon.csv')

df=df1.dropna(axis=0, how="any")

text_data = df['Text_of_Speech']
text_data = text_data.fillna('')
text_data = text_data.str.lower()

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

text_data = text_data.apply(remove_punctuation)
text_data = text_data.apply(word_tokenize)
stop_words = set(stopwords.words('english')) 

def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

text_data = text_data.apply(remove_stopwords)
      
def remove_non_alphabetic(text):
    return [word for word in text if word.isalpha()]

text_data = text_data.apply(remove_non_alphabetic)
text_data = text_data.apply(' '.join)
text_data_list = text_data.tolist()

#stemming

stemmer = PorterStemmer()
stemmed_text_data_list = [[stemmer.stem(word) for word in sentence.split()] for sentence in text_data_list]

#lemmatization

lemmatizer = WordNetLemmatizer()

lemmatized_text_data_list = [[lemmatizer.lemmatize(word) for word in sentence.split()] for sentence in text_data_list]

lemmatized_text_data = [' '.join(sentence) for sentence in lemmatized_text_data_list]

tfidf_vectorizer = TfidfVectorizer(max_features=5000)  

tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_text_data)


target = df['Emotion']


X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, target, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')

report = classification_report(y_test, y_pred)
print(report)

Accuracy: 0.49221183800623053
              precision    recall  f1-score   support

         JOY       0.53      0.59      0.56        78
     NEUTRAL       0.42      0.49      0.45        95
    OPTIMISM       0.48      0.51      0.50        92
       UPSET       0.72      0.32      0.44        56

    accuracy                           0.49       321
   macro avg       0.54      0.48      0.49       321
weighted avg       0.52      0.49      0.49       321

